# SO3 to SO3 Damping

In [ ]:
using PBDS, StaticArrays, LinearAlgebra, BenchmarkTools, Rotations, Random

## Setup

### SO3 Damping

In [ ]:
PBDS.default_coord_rep(::Damping{<:Identity{SO3,SO3}}) = ChartRep()
PBDS.metric_chart(xn, task::Damping{<:Identity{SO3,SO3}}, CN::Chart{ExpMap0,SO3}) =
    default_metric(xn, task, CN)
PBDS.potential_chart(xn, task::Damping{<:Identity{SO3,SO3}}, CN::Chart{<:ExpMap,SO3}) = 0.

function PBDS.dissipative_forces_chart(xn, vn, task::Damping{<:Identity{SO3,SO3}}, CN::Chart{<:ExpMap,SO3})
    xne, vne = chart_to_emb_differential(xn, vn, CN)
    Fne = -vne
    ∂xne_∂xn = chart_to_emb_jacobian(xn, CN)
    Fn = ∂xne_∂xn'*Fne
end
PBDS.weight_metric_chart(xn, vn, task::Damping{<:Identity{SO3,SO3}}, CN::Chart{ExpMap0,SO3}) =
    default_weight_metric(xn, vn, task, CN)
PBDS.home_task_chart(task::Damping{<:Identity{SO3,SO3}}, CN::Chart{<:ExpMap,SO3}) = Chart{ExpMap0,SO3}()

## Point Acceleration

In [ ]:
# Initial state
skew(w) = SA[0.   -w[3]  w[2]
             w[3]  0.   -w[1]
            -w[2]  w[1]  0.]

Random.seed!(8)
R = rand(RotMatrix{3}).mat
ω = R*SA[0.,0.,100.]
xme = reshape(R, Size(9))
vme = reshape(skew(ω)*R, Size(9))

C0 = Chart{PBDS.ExpMap0,SO3}()

M = SO3
tasks, CNs = TaskList(), ChartList()

N = SO3
CN = choose_chart_emb(xme, C0)
# CN = Chart{ExpMap0,N}()
push!(tasks, Damping(Identity{M,N,Float64}()))
push!(CNs, CN)

CM = CN
robot_coord_rep = EmbRep()
σxddot, = multiple_task_acceleration(xme, vme, tasks, CM, CNs, robot_coord_rep)

## Single Trajectory

In [ ]:
using Plots, Makie, Observables, ProgressMeter

In [ ]:
Time = 4
dt = 0.01

traj = propagate_tasks(xme, vme, tasks, CM, CNs, Time, dt, robot_coord_rep, log_tasks=true)
traj.vm[end]

In [ ]:
function frot(i)
    q = UnitQuaternion(RotMatrix(traj.xm[i]))
    qrot = Quaternion(q.x, q.y, q.z, q.w)
end

iobs = Observable(1)
ax_size, plot_size = 2, 800
limits = FRect3D((-ax_size, -ax_size, -ax_size), (2*ax_size, 2*ax_size, 2*ax_size))
scene = Scene(resolution = (plot_size, plot_size))
widths = SA[1.,2.,3.]
rect = Rect(Vec(-widths./2), Vec(widths))
rect_mesh = mesh!(scene, rect, color = :orange; limits)
Makie.rotate!(rect_mesh[end], frot(1))

axis = scene[Axis]
axis.showaxis = false
display(scene)

In [ ]:
function record_scene(scene, filename, iobs, N, framerate=60)
    p = Progress(N)
    record(scene, filename, 1:N) do i
        Makie.rotate!(rect_mesh[end], frot(i))
        next!(p)
    end
    display("text/html", html_video(filename))
end

filename = "SO3ToSO3_Damping.mp4"
record_scene(scene, filename, iobs, length(traj.xm))

In [ ]:
C0 = Chart{ExpMap0,SO3}()
Cx = Chart{ExpMapX,SO3}()
Cy = Chart{ExpMapY,SO3}()
Cz = Chart{ExpMapZ,SO3}()

In [ ]:
ω_axes, cs = [], []
for i in 1:length(traj.xm)
    xm_test = traj.xm[i]
    vm_test = traj.vm[i]
    R_test = reshape(xm_test, Size(3,3))
    Rdot_test = reshape(vm_test, Size(3,3))
    ωskew_test = Rdot_test*R_test'
    ω_test = PBDS.unskew(ωskew_test)
    ωmag = norm(ω_test)
    ω_axis = ωmag > 0 ? ω_test/ωmag : (@SVector zeros(3))
    push!(ω_axes, ω_test/norm(ω_test))
    C = traj.CM[i]
    C == C0 && push!(cs, -0.1)
    C == Cx && push!(cs, 0.)
    C == Cy && push!(cs, 0.1)
    C == Cz && push!(cs, 0.2)
end

In [ ]:
using Plots
scene = Plots.plot(getindex.(ω_axes,1), lab = "Spin axis x")
Plots.plot!(scene, getindex.(ω_axes,2), lab = "Spin axis y")
Plots.plot!(scene, getindex.(ω_axes,3), lab = "Spin axis z")
Plots.plot!(scene, cs; legend=:outerright, lab = "Chart")